In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
from lo.sdk.api.acquisition.io.open import open
from lo.sdk.api.acquisition.data.decode import SpectralDecoder
from lo.sdk.api.acquisition.data.formats import LORAWtoRGB8

In [ ]:
directory = '/Volumes/LaCie/Living_Optics_hyperspec_Josh/Wytham_raw_data/'

factory_cali = os.path.expanduser('~/Desktop/Rachael_DRAGNet_Project/Metadata/LO1_calibration_files/8_5mm')

date_file = glob.glob(directory + '*')


In [ ]:
data_folders = defaultdict(list)


for i in date_file:
    label = i.split('/')[-1]
    blocks = glob.glob(i + '/*BLOCK*')
    
    for j in blocks:
        treatments = glob.glob(j + '/*')
        data_folders[label].append(treatments)


print(data_folders)

excel = pd.read_csv('/Volumes/LaCie/Living_Optics_hyperspec_Josh/RT_hyperspectral_data_log_2024.csv', nrows=50)
weather_excel = excel['Weather']
date_excel = excel[excel.columns[1]].values
treatment_excel = excel['Treatment']




In [ ]:
metadata_dict = defaultdict(list)

for date in data_folders:
    blocks = data_folders[date]
    for block in blocks:
        for treatment in block:
            calibration = glob.glob(treatment + '/Calibration/*.loraw')
            decode = SpectralDecoder.from_calibration(factory_cali, calibration[0])
            images = glob.glob(treatment + '/Data/*.loraw')
            for image in images:
                with open(image, mode='r') as f:
                    for frame in f:
                        metadata, scene_view, spectra = decode(frame, scene_decoder=LORAWtoRGB8)
                        date = image.split('/')[-5].replace('_','/')
                        metadata_dict['Site'].append(image.split('/')[4].split('_')[0])
                        metadata_dict['Date'].append(date)
                        metadata_dict['Block'].append(image.split('/')[-4].split('_')[-1])
                        treatment = image.split('/')[-3]
                        metadata_dict['Treatment'].append(treatment)
                        if date == '2024/06/25' and treatment == 'NPK_DIST':
                            time = image.split('/')[-1].split('-')[1]
                        else:
                            time = image.split('/')[-1].split('-')[2]
                        metadata_dict['Time'].append(time[:2] + ':' + time[2:4] + ':' + time[4:6])
                        found = False
                        i = 0
                        while found == False:
                            if date_excel[i][0:2] == date[-2:] and treatment_excel[i] == metadata_dict['Treatment'][-1]:
                                metadata_dict['Weather'].append(weather_excel[i])
                                i = 0
                                found = True
                            else:
                                i += 1
                            if i == 50:
                                metadata_dict['Weather'].append('Not Found')
                                found = True
                        if image.split('-')[-1] == 'single.loraw':
                            metadata_dict['Video'].append('Single')
                        else:
                            metadata_dict['Video'].append('Video')
                        metadata_dict['Frames'].append(len(f))
                        for v in vars(metadata):
                            if v == 'wavelengths' or v == 'sampling_coordinates':
                                continue
                            else:
                                metadata_dict[v].append(getattr(metadata, v))
                        break

 
df = pd.DataFrame(metadata_dict)
df.to_csv('Meta_Data.csv', index=False)


In [ ]:
print(metadata_dict)